In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_csv("../Clean_csvs/age_family_region_clean.csv")
df2 = pd.read_csv("../Clean_csvs/project_3_political_income_gap.csv")
df3 = pd.read_csv("../Clean_csvs/project_3_relationship_len_up_to_wv6.csv")

In [2]:
df12 = df1.merge(df2, on="caseid_new")

In [3]:
df12 = df12[["caseid_new", "age_when_met", "family_size", "region", "metro", "same_political", "income_gap"]]
df12.head()
# df12.shape
df12[df12["caseid_new"] == 745435]


,caseid_new,age_when_met,family_size,region,metro,same_political,income_gap
470,745435,42.0,NaN,1,1,0,1.0


In [44]:
df123 = df12.merge(df3, on="caseid_new")
df123.head()
# df123.replace(to_replace="NAN", value="")
df123['family_size'].fillna(round(df123['family_size'].mean()), inplace=True)
df123['relationship_len'].fillna(round(df123['relationship_len'].mean()), inplace=True)
df123['age_when_met'].fillna(round(df123['age_when_met'].mean()), inplace=True)
df123['region'].fillna(df123['region'].mode(), inplace=True)
df123['metro'].fillna(df123['metro'].mode(), inplace=True)
df123['same_political'].fillna(df123['same_political'].mode(), inplace=True)
df123['income_gap'].fillna(round(df123['income_gap'].mean()), inplace=True)
df123['partner_deceased'].fillna(df123['partner_deceased'].mode(), inplace=True)

# df123.shape


In [45]:
df123['income_gap'].value_counts()

1.0    2635
0.0     374
Name: income_gap, dtype: int64

In [5]:
df123[df123["caseid_new"] == 745435]
# df123 = df123[df123.family_size != "NaN"]
# df123.shape

,caseid_new,age_when_met,family_size,region,metro,same_political,income_gap,relationship_len,partner_deceased,relationship_quality_w1
470,745435,42.0,4.0,1,1,0,1.0,22.0,0,good


In [46]:
df123.relationship_quality_w1 = df123.relationship_quality_w1.map({'very poor':0, 'poor':1, 'fair':2, 'good':3, 'excellent':4})
df123['relationship_quality_w1'].fillna(df123['relationship_quality_w1'].mode()[0], inplace=True)


In [50]:
df123.head()
# np.where(df123.isnull != False and != True)
df123.isnull().sum()

# np.where(df123['income_gap'] == "")

caseid_new                 0
age_when_met               0
family_size                0
region                     0
metro                      0
same_political             0
income_gap                 0
relationship_len           0
partner_deceased           0
relationship_quality_w1    0
dtype: int64

In [67]:
from sklearn.model_selection import train_test_split

selected_features = df123[["age_when_met", "family_size", "region", "metro", "same_political", "income_gap", "partner_deceased", "relationship_quality_w1"]]
y = df123["relationship_len"]

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=0)

In [68]:
# for index, row in df123.iterrows():
#     if row.empty == True:
#         print(row["caseid_new"])
# df123 = df123.astype('float64')
# np.where(np.isnan(df123))
df123.reset_index(drop=True)

,caseid_new,age_when_met,family_size,region,metro,same_political,income_gap,relationship_len,partner_deceased,relationship_quality_w1
0,22526,45.0,0.0,2,1,1,1.0,12.00,0,3.0
1,23286,19.0,0.0,1,1,1,1.0,13.00,0,3.0
2,26315,23.0,1.0,0,1,1,1.0,8.00,0,3.0
3,28536,41.0,1.0,2,1,1,1.0,17.00,0,3.0
4,29584,28.0,0.0,0,1,1,1.0,35.00,0,3.0
5,31456,42.0,8.0,0,1,0,1.0,9.00,0,4.0
6,32656,36.0,0.0,3,1,1,1.0,32.00,0,4.0
7,33536,38.0,0.0,1,1,1,0.0,18.00,0,4.0
8,34341,18.0,0.0,2,1,0,1.0,19.00,0,4.0
9,35653,51.0,0.0,1,1,1,1.0,14.00,0,4.0


In [88]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

model1 = RandomForestRegressor(max_depth=5, random_state=0)
model1.fit(X_train, y_train)
print(model1.feature_importances_)
model1.predict(X_test)

[0.66811028 0.07737069 0.04101376 0.01470178 0.00520403 0.00614778
 0.11086194 0.07658973]


C:\Users\megam\Anaconda3\envs\PythonData\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([22.07509677, 20.18494814, 13.62079208, 14.06810511, 17.79216448,
       18.79867544, 24.82221902, 31.30999919, 22.50128215, 26.284117  ,
       21.000608  , 27.83596865, 22.91068541, 24.07547595, 28.30993535,
       26.55455985, 25.04219177, 20.52702335, 26.73152636, 24.35774445,
       14.32939082, 24.49765467, 19.78380761, 22.20320722,  9.48029029,
       26.56096845, 14.22327143, 24.49765467, 15.31017319, 28.60988351,
       21.15024897, 26.73152636,  9.48029029, 29.8712368 , 26.55455985,
        8.51655923,  7.96089735, 26.10817127, 17.13803883, 14.88127736,
       18.26972316, 15.59980429, 37.40329787, 21.97266645, 21.36864726,
       29.4437002 , 26.44006091, 13.74950013, 18.12532997,  7.46303827,
       29.47801799, 30.11750928, 29.8712368 , 25.97373109, 15.93235956,
       24.79895325, 23.19096809, 10.09414405, 22.91068541, 25.25435165,
       24.49765467, 28.56873448, 17.89507086, 21.65407422, 28.18459285,
       20.09064853, 18.93335492, 28.46910353, 28.99365972, 23.46

In [89]:
# from sklearn.ensemble import RandomForestClassifier
# # Creating a random forest classifier
# model2 = RandomForestClassifier(n_jobs=2, random_state=0)
# # Training classifier
# model2.fit(X_train, y_train)

In [95]:
# Finding important features

num_important = (model1.feature_importances_).astype(str)
features = X_test.columns

def myfunc(a, b):
    return [round(float(b)*100, 2)] + [a]

def to_percentage(score):
    return f'{round(score * 100, 2)}%'

important_features_map = map(myfunc, features, num_important)
important_features = dict(important_features_map)
 
sorted_important_features = sorted(important_features.items(), reverse=True)

print(f"Training Data Score: {to_percentage(model1.score(X_train, y_train))}")
print(f"Testing Data Score: {to_percentage(model1.score(X_test, y_test))}")
print("-----------------------------------------------------")
print(f"Most important features:\n{sorted_important_features[:4]}")

Training Data Score: 21.15%
Testing Data Score: 14.78%
-----------------------------------------------------
Most important features:
[(66.81, 'age_when_met'), (11.09, 'partner_deceased'), (7.74, 'family_size'), (7.66, 'relationship_quality_w1')]


In [ ]:
# Finding important features

num_important = (model2.feature_importances_).astype(str)
features = X_test.columns

def myfunc(a, b):
    return [round(float(b)*100, 2)] + [a]

def to_percentage(score):
    return f'{round(score * 100, 2)}%'

important_features_map = map(myfunc, features, num_important)
important_features = dict(important_features_map)
 
sorted_important_features = sorted(important_features.items(), reverse=True)

print(f"Training Data Score: {to_percentage(model2.score(X_train, y_train))}")
print(f"Testing Data Score: {to_percentage(model2.score(X_test, y_test))}")
print("-----------------------------------------------------")
print(f"Most important features:\n{sorted_important_features[:4]}")